In [ ]:
import dicom
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import glob
import re

from skimage.transform import resize

from sklearn.model_selection import train_test_split
from dicom.contrib.pydicom_PIL import show_PIL
from keras.preprocessing.image import img_to_array, load_img

# only use GPUID == 1
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [ ]:
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image       import ImageDataGenerator
from keras.models                    import Sequential, Model, Input
from keras.layers                    import Dense, Flatten, Dropout
from keras.optimizers                import SGD, RMSprop, Adam
from keras.callbacks                 import EarlyStopping
from keras.utils                     import np_utils
from keras                           import backend as K

In [ ]:
dirPath = '/enc_data/eddata/pacemaker'
dataPath = os.path.join(dirPath, 'organized-data')
allPatients = os.listdir(dataPath)
print('Number of patients: {}'.format(len(allPatients)))

In [ ]:
number = 100

In [ ]:
pacemakerImgs = set()
ratios = []

for image_path in glob.glob(os.path.join(dirPath,"png/full_image/*.png"))[:number]:
    patientid, clipnum = re.split('image\/(\d*)_(\d*)-', image_path)[1:3]
    pacemakerImgs.add((patientid, clipnum))
    
    img = plt.imread(image_path)
    ratio = img.shape[1]/float(img.shape[0])
    ratios.append(ratio)

plt.hist(ratios)
plt.show()

In [ ]:
# Set .85 ratio
ratio = .85

In [ ]:
# Scale images with current shape to new ratio
def getScaledDims(currshape, ratio):
    w,h = currshape
    currratio = h/float(w)
    
    if currratio > ratio: # Height is larger
        new_h = int(w*ratio)
        return (w,new_h)
    else:
        new_w = int(h/ratio)
        return (new_w, h)

In [ ]:
plt.imshow(img)
plt.show()
print('{} current ratio: {}'.format(img.shape, img.shape[1]/float(img.shape[0])))

In [ ]:
newdims = getScaledDims(img.shape, ratio)
print(newdims, newdims[1]/float(newdims[0]))

In [ ]:
def cropCenter(img,cropy,cropx):
    y, x = img.shape
    startx = x//2-(cropx//2)
    starty = y//2-(cropy//2)    
    return img[starty:starty+cropy, startx:startx+cropx, ]

In [ ]:
croppedimg = cropCenter(img, newdims[0], newdims[1])
print(croppedimg.shape, newdims)
plt.imshow(croppedimg)
plt.show()

In [ ]:
def loadImage(image_path, ratio):
    img = plt.imread(image_path)
    currshape = img.shape
    cropy, cropx = getScaledDims(currshape, ratio)
    newimg = cropCenter(img,cropy,cropx)
    return newimg

In [ ]:
ratio = 0.85
x_True = [loadImage(image_path, ratio) for image_path in glob.glob(os.path.join(dirPath,"png/full_image/*.png"))[:number]]

In [ ]:
# Get the most common dimensions and reshape each image to that dimension
from scipy.stats import mode

shapes = np.array([np.array(img.shape) for img in x_True])
imgdims = (mode(shapes[:,0]).mode[0], mode(shapes[:,1]).mode[0])
imgdims

In [ ]:
imgdims + (1,)

In [ ]:
# Five examples
imgdims = (225, 255)

for img in x_True[:2]:
    print(img.shape)
    plt.imshow(img)
    plt.show()
    newimg = resize(img, imgdims)
    newimg_3d = np.empty(imgdims + (3,))
    for i in range(3):
        newimg_3d[:,:,i] = newimg
    print(newimg_3d.shape)
    plt.imshow(newimg_3d[:,:,1])
    plt.show()

##### Generate cropped image arrays for each class and resize them to the most common dimensions

In [ ]:
from scipy.stats import mode

def loadImage(image_path, ratio):
    img = plt.imread(image_path)
    currshape = img.shape
    cropx, cropy = getScaledDims(currshape, ratio)
    newimg = cropCenter(img,cropx,cropy)
    return newimg

def resizeImage(image, dims):
    newimg = resize(image, dims)
    
    # Convert from 1 channel to 3 channels
    newimg_3d = np.empty(imgdims + (3,))
    for i in range(3):
        newimg_3d[:,:,i] = newimg
        
    return newimg_3d

def loadTrueClass(path, number, dims):
    """ Path: Path to .png images
        Number: Number of images to run
        Dims: (height, width) of the final images that will be fed into the model """
    
    # Crop images to ratio specified.
    ratio = dims[1]/float(dims[0])
    croppedImgs = [loadImage(image_path, ratio) for image_path in glob.glob(path)[:number]]
    
    # Resize all images to that dims
    x_True = np.array([resizeImage(img, dims) for img in croppedImgs])
    return x_True

In [ ]:
pacemakerPath = os.path.join(dirPath,"png/full_image/*.png")
x_True = loadTrueClass(pacemakerPath, 50, (225, 255))
x_True.shape

# Data should match shape format: (samples, height, width, channels) 

In [ ]:
len(glob.glob(pacemakerPath))

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255, horizontal_flip = True, vertical_flip = True)
train_datagen.fit(x_True)
# generator = train_datagen.flow(X_train, y_train, batch_size=32)
